### Thermo property packages

A Thermo object defines a thermodynamic property package. To build a Thermo object, we must first define all the chemicals involed. The following example will show how this can be done through the creation of a property package for the co-production of biodiesel and ethanol from lipid-cane [[1]](#References):

We can first start by defining the common chemicals already in the data base:

In [1]:
import thermosteam as tmo
Biodiesel = tmo.Chemical('Biodiesel', search_ID='Methyl oleate')
chemicals = tmo.Chemicals(['Water', 'Methanol', 'Ethanol', 'Glycerol', 'Glucose',
                           'Sucrose', 'H3PO4', 'CO2', 'Octane', Biodiesel])
(Water, Methanol, Ethanol,
 Glycerol, Glucose, Sucrose,
 H3PO4, CO2, Octane, Biodiesel) = chemicals

We can assume that CO2 gas will always remain a gas in the process by setting the state:

In [2]:
CO2.at_state(phase='g')

Similarly, we can assume glucose, sucrose, and phosphoric acid all remain as solids:

In [6]:
Glucose.at_state(phase='s')
Sucrose.at_state(phase='s')
H3PO4.at_state(phase='s')

Now we can define the solids in the process (both soluble and insoluble). We can use the `Chemical.blank` method to create a blank Chemical object and add the thermo models ourselves:

In [7]:
def create_new_chemical(ID, phase='s', **constants):
    # Create a new solid chemical without any data
    solid = tmo.Chemical.blank(ID, phase=phase, **constants)
    
    # Add chemical to the Chemicals object
    chemicals.append(solid)
    
    return solid

Ash = create_new_chemical('Ash')
Cellulose = create_new_chemical('Cellulose')
Hemicellulose = create_new_chemical('Hemicellulose')
Flocculant = create_new_chemical('Flocculant')
Lignin = create_new_chemical('Lignin')
Solids = create_new_chemical('Solids')
DryYeast = create_new_chemical('DryYeast', CAS='Yeast')
CaO = create_new_chemical('CaO', MW=56.0774)
HCl = create_new_chemical('HCl', MW=36.46094)
NaOH = create_new_chemical('NaOH', MW=39.997109)
NaOCH3 = create_new_chemical('NaOCH3', MW=54.023689)

Note that we are still missing the lipid, modeled as Triolein. However, Triolein is not in the data bank, so let's make it from scratch:

In [8]:
Lipid = create_new_chemical(
    'Lipid',
    phase='l',
    Hc=35099.6e3, # https://webbook.nist.gov/cgi/cbook.cgi?ID=C122327&Mask=2
    Hf=-2193.7e3, 
    MW=885.432,
)

All what is left is to fill the chemical properties. This done through the `add_model` method of the chemical model handles. Let's begin with the solids using data from [[1-4]](#References):

In [9]:
from thermosteam import functional as fn
insoluble_solid_molar_volume = fn.rho_to_V(rho=1540, MW=1.)

# Insolubles occupy a significant volume
insoluble_solids = (Ash, Cellulose, Hemicellulose, Flocculant, Lignin, Solids, DryYeast)

# Solubles don't occupy much volume
soluble_solids = (CaO, HCl, NaOH, Glucose, Sucrose) 

for chemical in insoluble_solids:
    chemical.V.add_model(insoluble_solid_molar_volume)

for chemical in soluble_solids:
    V = fn.rho_to_V(rho=1e5, MW=chemical.MW)
    chemical.V.add_model(V, top_priority=True)

# Assume sodium methoxide has same properities as methanol
# except for molecular weight
NaOCH3.fill(chemicals.Methanol)

# Add constant models for molar heat capacity of solids
Ash.Cn.add_model(0.09 * 4.184) 
CaO.Cn.add_model(1.02388 * 56.0774) 
Cellulose.Cn.add_model(1.364) 
Hemicellulose.Cn.add_model(1.364)
Flocculant.Cn.add_model(4.184)
Lignin.Cn.add_model(1.364)
Solids.Cn.add_model(1.100)

# Set heats of formation and combustion (for those that need it)
Hemicellulose.Hc = 17e3
Cellulose.Hc = 17e3
Lignin.Hc = 21e3
Glucose.Hc = 2.8e6
Sucrose.Hc = 5.7e6
Lipid.Hc = 36e6

Ethanol.Hf = -277.690e3
Water.Hf = -241.820e3
CO2.Hf = -393.520e3 # in gas
Glucose.Hf = -1274e3
Sucrose.Hf = -2221.2e3
Octane.Hf = -250e3
H3PO4.Hf = -1271.66e3
Lipid.Hf = -2.1937e3
Biodiesel.Hf = -0.72764e3
Glycerol.Hf = -0.6696e3

We don't care much about the rest of the properties (e.g. thermal conductivity), so we can default them to the values of water:

In [10]:
for chemical in chemicals:
    chemical.default()

Once the chemicals are finalized, we can compile them into a Thermo object:

In [13]:
thermo = tmo.Thermo(chemicals)
thermo

Thermo(chemicals=CompiledChemicals([Water, Methanol, Ethanol, Glycerol, Glucose, Sucrose, H3PO4, CO2, Octane, Biodiesel, Ash, Cellulose, Hemicellulose, Flocculant, Lignin, Solids, DryYeast, CaO, HCl, NaOH, NaOCH3, Lipid]), mixture=IdealMixture(...), Gamma=DortmundActivityCoefficients, Phi=IdealFugacityCoefficients, PCF=IdealPoyintingCorrectionFactors)

Note that the Thermo object, by default, chooses ideal mixing rules for estimating mixture properties, and neglects excess properties in the calculation of enthalpy and entropy:

In [14]:
thermo.mixture

IdealMixture(chemicals=CompiledChemicals([Water, Methanol, Ethanol, Glycerol, Glucose, Sucrose, H3PO4, CO2, Octane, Biodiesel, Ash, Cellulose, Hemicellulose, Flocculant, Lignin, Solids, DryYeast, CaO, HCl, NaOH, NaOCH3, Lipid]), rigorous_energy_balance=True, include_excess_energies=False)

It is possible to implement a your own Mixture object so long as it implements the same methods and attributes as an `IdealMixture` class. This is out the scope of ordinary users, but you can request help by posting in https://github.com/yoelcortes/thermosteam.

A thermo object contains ActivityCoefficients, FugacityCoefficients, and PoyintingCorrectionFactors subclasses to define fugacity estimation methods. By default, the Dortmund modified UNIFAC method for estimating activities is selected, while ideal values for (vapor phase) fugacity coefficients and poyinting correction factos are selected.

### References

<a id='References'></a>

1. Huang, H., Long, S., & Singh, V. (2016) “Techno-economic analysis of biodiesel and ethanol co-production from lipid-producing sugarcane” Biofuels, Bioproducts and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640

2. Hatakeyama, T., Nakamura, K., & Hatakeyama, H. (1982). Studies on heat capacity of cellulose and lignin by differential scanning calorimetry. Polymer, 23(12), 1801–1804. https://doi.org/10.1016/0032-3861(82)90125-2

3. Thybring, E. E. (2014). Explaining the heat capacity of wood constituents by molecular vibrations. Journal of Materials Science, 49(3), 1317–1327. https://doi.org/10.1007/s10853-013-7815-6

4. Murphy W. K., and K. R. Masters. (1978). Gross heat of combustion of northern red oak (Quercus rubra) chemical components. Wood Sci. 10:139-141.